In [2]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
client = MistralClient()

In [3]:
from duckduckgo_search import DDGS

def web_search(query: str, max_results: int = 10) -> str:
    """
    Searched the web for the provided query, and returns the title, URL and description of the results.
    Search results are separated by: =====

    Example of two search results:

    Title: Welcome to My Site
    URL: http://www.mysite.xyz
    Description: This is my private website, see my stuff here
    =====
    Title: Grapes Online
    URL: http://www.grapes.com
    Description: This is the number one site for grapes fans and lovers
    """
    if max_results < 1: max_results = 1
    elif max_results > 10: max_results = 10

    ddgs = DDGS()
    outputs = list()
        
    results = ddgs.text(query, max_results=max_results)

    try:
        for result in results:
            url = result['href']
            if url is None:
                continue
            body = result['body']
            title = result['title'] or ''
            outputs.append(f"Title: {title}\nURL: {url}\nDescription: {body}\n")
    
    except Exception as e:
        return f"Error while searching the web: {e}"

    if outputs:
        return '\n=====\n'.join(outputs)
    else:
        return "No results"
    
tools_def = {
    web_search: [("query", {"type": "string", "description": "The query to search on the web"}, True),
                 ("max_results", {"type": "number", "description": "Maximal number of results to retrieve. Must be between 1 and 10, default is 10."}, False)]
}

def build_tools():
    tools = list()
    for func, v in tools_def.items():
        params = {}
        required = []
        for p in v:
            params[p[0]] = p[1]
            if p[2]: 
                required.append(p[0])

        tools.append({
            "type": "function",
            "function": {
                "name": func.__name__,
                "description": func.__doc__,
                "parameters": {
                    "type": "object",
                    "properties": params
                },
                "required": required
            }
        })
    return tools 

In [8]:
messages = [
    ChatMessage(role="system", content="You are an AI assistant named Bruno"),
    ChatMessage(role="user", content="Search the ewb for the meaning of your name")
]
response = client.chat(model='mistral-small-latest', messages=messages)

response

ChatCompletionResponse(id='b210ef3aa929490cbebdb158bc52f2a9', object='chat.completion', created=1711645353, model='mistral-small-latest', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='Bruno is a male given name of Germanic origin. The name Bruno is derived from the Old High German word "brun" or "bruno," which means "brown" or "shield." It was a common name among the Germanic peoples in the early Middle Ages, and it was also the name of several saints, including Saint Bruno of Cologne, the founder of the Carthusian Order. In modern times, the name Bruno is still used in many countries, especially in German-speaking countries, Italy, and the United States. According to the Social Security Administration, Bruno was ranked as the 605th most popular boy\'s name in the United States in 2020.', name=None, tool_calls=None), finish_reason=<FinishReason.stop: 'stop'>)], usage=UsageInfo(prompt_tokens=22, total_tokens=173, completion_tokens=151))